# In Class Lab 1

### Due by midnight, thursday in your github repository 'Labs/Lab1' folder


In [1]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants - to deal with units

### Astropy Units:  https://docs.astropy.org/en/stable/units/index.html


## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [2]:
#git push into my repository into Lab 1
#compute with three different numbers

# 4.74*mu*Ro = VSLR + vsun

def VLSR(Ro, mu=6.379, vsun=12.24*u.km/u.s):
    """ This function will compute the velocity at the local standard of rest.
            VLSR = 4.74*mu*Ro - vsun
    Inputs: 
            Ro (astropy units kpc) Distance from the Sun to the Galactic Center
            mu is the proper motion of Sag A* (mas/yr)
                Default is from Reid & Brunthaler 2004
            vsun (astropy units km/s) the peculiar motion of the Sun in the v-dir (Schonrich+2010)
    
    Outputs: VLSR (astropy units km/s) the local standard of rest
    """
    VLSR = 4.74*mu*(Ro/u.kpc)*u.km/u.s - vsun
    return VLSR
    
    

In [3]:
#Different values of the distance to the Galactic Center
RoReid = 8.34*u.kpc #Reid+2014
RoAbuter = 8.178*u.kpc #Gravity Abuter+2019
RoSparke = 7.9*u.kpc #Sparke and Gallagher text

In [5]:
#Compute VLSR using Reid 2014
VLSR_Reid = VLSR(RoReid)
print(VLSR_Reid)

239.9320764 km / s


In [6]:
#Compute VLSR using Gravity Abuter 2019
VLSR_Abuter = VLSR(RoAbuter)
print(VLSR_Abuter)

235.03376988000002 km / s


In [8]:
#Compute VLSR using Sparke & Gallagher
VLSR_Sparke = VLSR(RoSparke)
print(VLSR_Sparke)

226.628034 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [18]:
#Orbital period = 2piR/velocity (t = d/v)

def TorbSun(Ro, Vc):
    """ Function that computes the orbital period of the Sun ; T = 2 pi R / V
    Inputs: 
        Ro: (astropy quantity) distance to the Galactic Center from the Sun (kpc)
        Vc: (astropy quantity) velocity of the Sun in the "V" direction (km/s)
    Outputs:
        T = (astropy quantity) orbital period (Gyr)
    """
    #Units are complicated here
    
    VkpcGyr = Vc.to(u.kpc/u.Gyr) #converting V to kpc/Gyr
    T = 2*np.pi*Ro/VkpcGyr #orbital period
    
    return T

In [19]:
VsunPec = 12.24*u.km/u.s #peculiar motion

In [20]:
Vsun = VLSR_Abuter + VsunPec #total motion of the Sun in "V" direction

In [23]:
#Compute the orbital period of the Sun
T_Abuter = TorbSun(RoAbuter, Vsun)
print(T_Abuter)
print(np.round(T_Abuter,3))

0.20318680562272234 Gyr
0.203 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [25]:
AgeUniverse = 13.8*u.Gyr
print(AgeUniverse/T_Abuter)
#we have gone around the MW around 68 times

67.91779593023313


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [26]:
#Fitting rotation curve simulation umbers: pm=0; starmass=82; dm=11
#Constant G units
print(const.G)

  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018


In [27]:
#G unit conversion
Grav = const.G.to(u.kpc**3/u.Gyr**2/u.Msun)
print(Grav)

4.498502151469554e-06 kpc3 / (Gyr2 solMass)


In [28]:
#Create a function for the mass profile
#Density profile  rho = VLSR^2/(4*pi*G*R^2)
#Mass (r) = Integrate rho dV
#   Integrate rho 4*pi*G*r^2*dr
#   Integrate VLSR^2 / (4*pi*G*r^2) * 4*pi*r^2 dr
#   Integrate VLSR^2 / G
#   VLSR^2/G *r

def massIso(r, VLSR):
    """ This function will compute the dark matter mass enclosed within a given distance r, assuming an Isothermal Sphere Model
        M(r) = VLSR^2/G *r
        Inputs:
                r: (astropy quantity) distance from the Galactic Center (kpc)
                VLSR: (astropy quantity) the velocity at the local standard of rest (km/s)
        Outputs:
                M: (astropy quantity) mass enclosed within r (Msun)
    """
    
    VLSRkpcGyr = VLSR.to(u.kpc/u.Gyr) #translating to kpc/Gyr
    M = VLSRkpcGyr**2/Grav *r #isothermal sphere mass profile - assume constant circular speed, assume VLSR same the whole time
    
    return M

In [31]:
#Compute mass enclosed within Ro (Gravity Collab)
mIsoSolar = massIso(RoAbuter, VLSR_Abuter)
print(mIsoSolar)
print(f"{mIsoSolar: .2e}") #f-string - how much mass it would have to have just within the enclosed radius

105038025820.79904 solMass
 1.05e+11 solMass


In [33]:
#Compute mass enclosed within 260kpc
mIso260 = massIso(260*u.kpc, VLSR_Abuter)
print(f"{mIso260:.2e}")

3.34e+12 solMass


In [34]:
#Compute mass enclosed within 220kpc
mIsoIAU = massIso(260*u.kpc, 220*u.kpc/u.s)
print(f"{mIsoIAU:.2e}")

2.79e+45 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [36]:
#Are the satellite galaxies around MW bound to our system and how much mass does there need to be to keep a satellite galaxy bound?
#Relate escape speed to potential, then can get profile

#Potential for a Hernquist Sphere
#   Phi = -G*M / (r+a)    plug this into vesc equation
#Escape speed becomes:
#   vesc^2 = 2*G*M / (r+a)

#Rearrange for M
#   M = vesc^2/2/G*(r+a)

def massHernVesc(vesc,r, a=30*u.kpc): #default variable always has to be last to run correctly
    """This function determines the total dark matter mass needed given an escape speed assuming a Hernquist profile
       M = vesc^2/2/G*(r+a)
    Inputs:
            vesc: (astropy quantity) escape speed or speed of satellite (km/s)
            r: (astropy quantity) distance from the Galactic Center (kpc)
            a: (astropy quantity) Hernquist scaling length (kpc) - default value of 30kpc
    Outputs:
            M: (astropy quantity) mass within r (Msun)
    """
    vescKpcGyr = vesc.to(u.kpc/u.Gyr) #translate to kpc/Gyr
    M = vescKpcGyr**2/2/Grav*(r+a)
    
    return M

In [37]:
Vleo = 196*u.km/u.s #speed of Leo I Sohn et al.
r = 260*u.kpc

In [39]:
MLeoI = massHernVesc(Vleo,r)
print(f"{MLeoI: .2e}") #this is the DM matter estimated for the MW based on how fast the satellite is moving

 1.30e+12 solMass


In [40]:
mIso260/MLeoI #estimates mass by a factor of 2.6

<Quantity 2.57842045>